In [1]:
import os
import sys
from os.path import join
import json

from tqdm import tqdm
import pandas as pd
import numpy as np
import nfl_data_py as nfl

ROOT_DIR = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.insert(0, os.path.join(ROOT_DIR,'py'))

import util

pd.set_option('display.max_rows',None)
pd.set_option('display.max_columns',None)

with open("paths.json", 'r') as f:
    paths = json.load(f)

RAW_DATA_PATH = paths['raw_data']
PROCESSED_DATA_PATH = paths['processed_data']

In [8]:
# Load the non-tracking data
df_game = pd.read_csv(join(RAW_DATA_PATH, "games.csv"))
df_play = pd.read_csv(join(RAW_DATA_PATH, "plays.csv"))
df_player_play = pd.read_csv(join(RAW_DATA_PATH, "player_play.csv"))
df_player = pd.read_csv(join(RAW_DATA_PATH, "players.csv"))
df_teams = nfl.import_team_desc()

# Uncamelcase the columns
df_game = util.uncamelcase_columns(df_game)
df_player = util.uncamelcase_columns(df_player)
df_play = util.uncamelcase_columns(df_play)
df_player_play = util.uncamelcase_columns(df_player_play)

# Add game_play_id to the play and player_play dataframes
df_play.insert(
    loc=2, 
    column='game_play_id', 
    value=df_play['game_id'].astype(str) + '_' + df_play['play_id'].astype(str)
)
df_player_play.insert(
    loc=2, 
    column='game_play_id',
    value=df_player_play['game_id'].astype(str) + '_' + df_player_play['play_id'].astype(str)
)

# Load nfl-data-py PBP data
pbp_cols = [
    'old_game_id_x','play_id','run_location','rusher_player_id','offense_formation',
    'offense_personnel','defenders_in_box','defense_personnel','desc'
]
rename_dict = {'old_game_id_x': 'game_id'}
rusher_positions = ['RB','TE','WR','FB','HB']
df_pbp = (
    nfl.import_pbp_data([2022])
    .query(
        'penalty == 0 and ' + \
        'fumble == 0  and ' +\
        'play_type == "run" and ' +\
        'qb_scramble == 0 and ' +\
        'qb_dropback == 0 and ' +\
        'qb_kneel == 0 and ' +\
        'qb_spike == 0 and ' +\
        '~play_type.isna() and ' +\
        'n_offense == 11 and ' +\
        'n_defense == 11 and ' +\
        'two_point_attempt == 0'
    )[pbp_cols]
    .rename(rename_dict, axis=1)
    .assign(
        game_id=lambda x: x.game_id.astype(int),
        play_id=lambda x: x.play_id.astype(int)
    ).merge(
        nfl.import_seasonal_rosters([2022],columns=['player_id','position']),
        left_on='rusher_player_id',
        right_on='player_id',
        how='left'
    ).query('position.isin(@rusher_positions)') # Remove QB runs (and one LB run )
    .drop(columns=['player_id','position','rusher_player_id'])
)

df_pbp = df_pbp[
    (df_pbp.desc.str.lower().str.contains('punt') == False) &
    (df_pbp.desc.str.lower().str.contains('penalty') == False) &
    (~df_pbp.offense_formation.isna()) &
    (~df_pbp.offense_personnel.isna()) &
    (~df_pbp.defenders_in_box.isna())
].drop(['desc'],axis=1)

2022 done.
Downcasting floats.


In [9]:
df_pbp.head()

,game_id,play_id,run_location,offense_formation,offense_personnel,defenders_in_box,defense_personnel
0,2022091107,68,left,SINGLEBACK,"1 RB, 2 TE, 2 WR",7.0,"3 DL, 4 LB, 4 DB"
1,2022091107,115,left,SINGLEBACK,"1 RB, 2 TE, 2 WR",7.0,"3 DL, 4 LB, 4 DB"
3,2022091107,275,right,SHOTGUN,"2 RB, 2 TE, 1 WR",7.0,"4 DL, 3 LB, 4 DB"
4,2022091107,330,right,I_FORM,"2 RB, 2 TE, 1 WR",8.0,"3 DL, 4 LB, 4 DB"
5,2022091107,391,right,SINGLEBACK,"1 RB, 2 TE, 2 WR",7.0,"3 DL, 4 LB, 4 DB"


In [12]:
game_ids = []
game_play_ids = []

for wk in tqdm(range(1,10), desc="filtering & standardizing tracking data"):

    # load tracking data, filtering down to run plays which are not a qb run
    df_tracking = (
        util.uncamelcase_columns(
            pd.read_csv(join(RAW_DATA_PATH, f'tracking_week_{wk}.csv'))
        ).merge(
            df_pbp,
            on=['game_id','play_id'],
            how='left'
        ).dropna(subset=['run_location'])
    )
    if 'week' not in df_tracking.columns:
        df_tracking.insert(3,'week',wk)

    # standardize direction to be offense moving right
    df_tracking, df_play_wk = util.standardize_direction(df_tracking, df_play)

    # Create single unique tracking data key
    df_tracking.insert(
        loc=2, 
        column='game_play_id', 
        value=df_tracking['game_id'].astype(str) + '_' + df_tracking['play_id'].astype(str) 
    )

    if df_play_wk.query('~((play_nullified_by_penalty == "N") and (penalty_yards != 0))').shape[0] > 0:
        raise ValueError('There are plays that were nullified by penalty or had penalty yards')

    df_tracking = df_tracking.merge(df_player[['nfl_id','position']], on='nfl_id', how='left')

    # add play info to tracking data
    df_tracking = df_tracking.merge(
        df_play_wk[['game_play_id','possession_team','absolute_yardline_number','yards_to_go']], 
        on=['game_play_id'], 
        how='left'
    )
    df_tracking['offense'] = df_tracking['club'] == df_tracking['possession_team']
    df_tracking['defense'] = ~df_tracking['offense'] & (df_tracking['club'] != 'football')
    df_tracking.drop(columns=['possession_team'], inplace=True)

    # create features of distance of players to ball
    tracking_ball = df_tracking[['game_play_id','frame_id','x','y','club']].query('club=="football"').copy()
    tracking_ball.rename(columns={'x':'ball_x','y':'ball_y'}, inplace=True)
    df_tracking = df_tracking.merge(
        tracking_ball[['game_play_id','frame_id','ball_x','ball_y']], 
        on=['game_play_id','frame_id'], 
        how='left'
    )
    del tracking_ball
    df_tracking['euclidean_dist_to_ball'] = np.sqrt((df_tracking['x'] - df_tracking['ball_x'])**2 + \
                                                    (df_tracking['y'] - df_tracking['ball_y'])**2)
    df_tracking['lateral_dist_to_ball'] = df_tracking['x'] - df_tracking['ball_x']
    df_tracking['vertical_dist_to_ball'] = df_tracking['y'] - df_tracking['ball_y']

    # Save the processed tracking data
    df_tracking.to_pickle(join(PROCESSED_DATA_PATH, f'tracking_week_{wk}.pkl'))
    df_play_wk.to_pickle(join(PROCESSED_DATA_PATH, f'play_week_{wk}.pkl'))

    game_ids.extend(df_tracking['game_id'].unique())
    game_play_ids.extend(df_tracking['game_play_id'].unique())

game_ids = list(set(game_ids))
game_play_ids = list(set(game_play_ids))

df_game = df_game.query('game_id.isin(@game_ids)')
df_player_play = df_player_play.query('game_play_id.isin(@game_play_ids)')

filtering & standardizing tracking data: 100%|██████████| 9/9 [02:14<00:00, 14.98s/it]


In [13]:
# store data to processed
df_game.to_pickle(join(PROCESSED_DATA_PATH, 'games.pkl'))
df_player_play.to_pickle(join(PROCESSED_DATA_PATH, 'player_play.pkl'))
df_player.to_pickle(join(PROCESSED_DATA_PATH, 'players.pkl'))
df_teams.to_pickle(join(PROCESSED_DATA_PATH, 'teams.pkl'))